In [1]:
# Installation
! pip install smolagents


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.4/148.4 kB 4.7 MB/s eta 0:00:00


In [2]:
import os
from getpass import getpass

# Set your Hugging Face token
os.environ["OPENAI_API_KEY"] = getpass("OPENAI_API_KEY")


OPENAI_API_KEY··········


# Text-to-SQL

In [3]:
!pip install smolagents python-dotenv sqlalchemy --upgrade -q

Then, we setup the SQL environment:

In [4]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    Float,
    insert,
    inspect,
    text,
)

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

def insert_rows_into_table(rows, table, engine=engine):
    for row in rows:
        stmt = insert(table).values(**row)
        with engine.begin() as connection:
            connection.execute(stmt)

table_name = "receipts"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("customer_name", String(16), primary_key=True),
    Column("price", Float),
    Column("tip", Float),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
]
insert_rows_into_table(rows, receipts)

### Build our agent

Now let’s make our SQL table retrievable by a tool.

The tool’s description attribute will be embedded in the LLM’s prompt by the agent system: it gives the LLM information about how to use the tool. This is where we want to describe the SQL table.

In [5]:
inspector = inspect(engine)
columns_info = [(col["name"], col["type"]) for col in inspector.get_columns("receipts")]

table_description = "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
print(table_description)

Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT


```text
Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT
```

Now let’s build our tool.
- A docstring with an `Args:` part listing arguments.
- Type hints on both inputs and output.

In [6]:
from smolagents import tool

@tool
def sql_engine(query: str) -> str:
    """
    Allows you to perform SQL queries on the table. Returns a string representation of the result.
    The table is named 'receipts'. Its description is as follows:
        Columns:
        - receipt_id: INTEGER
        - customer_name: VARCHAR(16)
        - price: FLOAT
        - tip: FLOAT

    Args:
        query: The query to perform. This should be correct SQL.
    """
    output = ""
    with engine.connect() as con:
        rows = con.execute(text(query))
        for row in rows:
            output += "\n" + str(row)
    return output

Now let us create an agent that leverages this tool.
*   We use the `CodeAgent`, which is smolagents’ main agent class: an agent that writes actions in code and can iterate on previous output according to the ReAct framework.
*   The model is the LLM that powers the agent system. `InferenceClientModel` allows you to call LLMs using OPENAI Inference API.

**Querying on receipts table**


In [7]:
from smolagents import CodeAgent, InferenceClientModel

model = InferenceClientModel(
    provider="openai",
    api_key=os.environ["OPENAI_API_KEY"],
    model_id="gpt-4o-mini"
)
agent = CodeAgent(
    tools=[sql_engine],
    model=model,
)
agent.run("Display whole table.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Display whole table.                                                                                            │
│                                                                                                                 │
╰─ InferenceClientModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = "SELECT * FROM receipts;"                                                                                
  table_data = sql_engine(query)                                                                                   
  print(table_data)                                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

(1, 'Alan Payne', 12.06, 1.2)
(2, 'Alex Mason', 23.86, 0.24)
(3, 'Woodrow Wilson', 53.43, 5.43)
(4, 'Margaret James', 21.11, 1.0)

Out: None

[Step 1: Duration 1.97 seconds| Input tokens: 2,068 | Output tokens: 72]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Directly printing the table data retrieved earlier                                                             
  final_answer(table_data)                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 
(1, 'Alan Payne', 12.06, 1.2)
(2, 'Alex Mason', 23.86, 0.24)
(3, 'Woodrow Wilson', 53.43, 5.43)
(4, 'Margaret James', 21.11, 1.0)

[Step 3: Duration 2.32 seconds| Input tokens: 6,972 | Output tokens: 264]

"\n(1, 'Alan Payne', 12.06, 1.2)\n(2, 'Alex Mason', 23.86, 0.24)\n(3, 'Woodrow Wilson', 53.43, 5.43)\n(4, 'Margaret James', 21.11, 1.0)"

In [9]:
from smolagents import CodeAgent, InferenceClientModel

model = InferenceClientModel(
    provider="openai",
    api_key=os.environ["OPENAI_API_KEY"],
    model_id="gpt-4o-mini"
)
agent = CodeAgent(
    tools=[sql_engine],
    model=model,
)
agent.run("Can you give me the name of the client who got the most expensive receipt?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of the client who got the most expensive receipt?                                      │
│                                                                                                                 │
╰─ InferenceClientModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = "SELECT customer_name FROM receipts ORDER BY price DESC LIMIT 1;"                                        
  most_expensive_client = sql_engine(query)                                                                        
  print(most_expensive_client)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

('Woodrow Wilson',)

Out: None

[Step 1: Duration 2.14 seconds| Input tokens: 2,080 | Output tokens: 104]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Woodrow Wilson")                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Woodrow Wilson

[Step 2: Duration 1.20 seconds| Input tokens: 4,360 | Output tokens: 155]

'Woodrow Wilson'

###  Table joins

**Created second table recording the names of waiters for each receipt_id called waiters**

In [11]:
table_name = "waiters"
waiters = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("waiter_name", String(16), primary_key=True),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "waiter_name": "Corey Johnson"},
    {"receipt_id": 2, "waiter_name": "Michael Wantts"},
    {"receipt_id": 3, "waiter_name": "Michael Watts"},
    {"receipt_id": 4, "waiter_name": "Margaret James"},
    {"receipt_id": 5, "waiter_name": "Paul Walton"},
    {"receipt_id": 6, "waiter_name": "Canary Mill"},
]
insert_rows_into_table(rows, waiters)

Since we changed the table, we update the `SQLExecutorTool` with this table’s description to let the LLM properly leverage information from this table.

In [12]:
updated_description = """Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:"""

inspector = inspect(engine)
for table in ["receipts", "waiters"]:
    columns_info = [(col["name"], col["type"]) for col in inspector.get_columns(table)]

    table_description = f"Table '{table}':\n"

    table_description += "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
    updated_description += "\n\n" + table_description

print(updated_description)

Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:

Table 'receipts':
Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT

Table 'waiters':
Columns:
  - receipt_id: INTEGER
  - waiter_name: VARCHAR(16)


**Querying on receipts and waiters table**


In [14]:
from smolagents import CodeAgent, InferenceClientModel

model = InferenceClientModel(
    provider="openai",
    api_key=os.environ["OPENAI_API_KEY"],
    model_id="gpt-4o-mini"
)
agent = CodeAgent(
    tools=[sql_engine],
    model=model,
)

agent.run("Display the joined table between receipts and waiters on receipt_id.")


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Display the joined table between receipts and waiters on receipt_id.                                            │
│                                                                                                                 │
╰─ InferenceClientModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = """                                                                                                      
  SELECT *                                                                                                         
  FROM receipts                                                                                                    
  JOIN waiters ON receipts.receipt_id = waiters.receipt_id;                                                        
  """                                                                                                              
  joined_table_result = sql_engine(query)                                                                          
  print(joined_table_result)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

(1, 'Alan Payne', 12.06, 1.2, 1, 'Corey Johnson')
(2, 'Alex Mason', 23.86, 0.24, 2, 'Michael Wantts')
(3, 'Woodrow Wilson', 53.43, 5.43, 3, 'Michael Watts')
(4, 'Margaret James', 21.11, 1.0, 4, 'Margaret James')

Out: None

[Step 1: Duration 4.35 seconds| Input tokens: 2,077 | Output tokens: 191]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_output = [                                                                                                 
      (1, 'Alan Payne', 12.06, 1.2, 1, 'Corey Johnson'),                                                           
      (2, 'Alex Mason', 23.86, 0.24, 2, 'Michael Wantts'),                                                         
      (3, 'Woodrow Wilson', 53.43, 5.43, 3, 'Michael Watts'),                                                      
      (4, 'Margaret James', 21.11, 1.0, 4, 'Margaret James'),                                                      
  ]                                                                                                                
                                                                                                                   
  final_answer(final_output)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: [(1, 'Alan Payne', 12.06, 1.2, 1, 'Corey Johnson'), (2, 'Alex Mason', 23.86, 0.24, 2, 'Michael 
Wantts'), (3, 'Woodrow Wilson', 53.43, 5.43, 3, 'Michael Watts'), (4, 'Margaret James', 21.11, 1.0, 4, 'Margaret 
James')]

[Step 2: Duration 5.13 seconds| Input tokens: 4,547 | Output tokens: 445]

[(1, 'Alan Payne', 12.06, 1.2, 1, 'Corey Johnson'),
 (2, 'Alex Mason', 23.86, 0.24, 2, 'Michael Wantts'),
 (3, 'Woodrow Wilson', 53.43, 5.43, 3, 'Michael Watts'),
 (4, 'Margaret James', 21.11, 1.0, 4, 'Margaret James')]

In [15]:
agent.run("Find customers who tipped below the average tip amount.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find customers who tipped below the average tip amount.                                                         │
│                                                                                                                 │
╰─ InferenceClientModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  average_tip_query = "SELECT AVG(tip) as avg_tip FROM receipts"                                                   
  average_tip_result = sql_engine(average_tip_query)                                                               
  print(average_tip_result)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

(1.9674999999999998,)

Out: None

[Step 1: Duration 1.73 seconds| Input tokens: 2,074 | Output tokens: 95]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(["Alan Payne", "Alex Mason", "Margaret James"])                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: ['Alan Payne', 'Alex Mason', 'Margaret James']

[Step 3: Duration 1.22 seconds| Input tokens: 6,823 | Output tokens: 239]

['Alan Payne', 'Alex Mason', 'Margaret James']

In [17]:
agent.run("Count how many receipts each waiter handled, sorted by most active.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Count how many receipts each waiter handled, sorted by most active.                                             │
│                                                                                                                 │
╰─ InferenceClientModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = """                                                                                                      
  SELECT customer_name, COUNT(receipt_id) AS receipt_count                                                         
  FROM receipts                                                                                                    
  GROUP BY customer_name                                                                                           
  ORDER BY receipt_count DESC;                                                                                     
  """                                                                                                              
  result = sql_engine(query)                                                                                       
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

('Woodrow Wilson', 1)
('Margaret James', 1)
('Alex Mason', 1)
('Alan Payne', 1)

Out: None

[Step 1: Duration 2.61 seconds| Input tokens: 2,077 | Output tokens: 133]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  active_waiters = [                                                                                               
      ('Woodrow Wilson', 1),                                                                                       
      ('Margaret James', 1),                                                                                       
      ('Alex Mason', 1),                                                                                           
      ('Alan Payne', 1)                                                                                            
  ]                                                                                                                
  summary = ', '.join([f"{name}: {count} receipt(s)" for name, count in active_waiters])                           
  final_answer(summary)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Woodrow Wilson: 1 receipt(s), Margaret James: 1 receipt(s), Alex Mason: 1 receipt(s), Alan Payne: 1 
receipt(s)

[Step 2: Duration 3.79 seconds| Input tokens: 4,418 | Output tokens: 311]

'Woodrow Wilson: 1 receipt(s), Margaret James: 1 receipt(s), Alex Mason: 1 receipt(s), Alan Payne: 1 receipt(s)'

**Above Query is wrongly executed by querying receipts table instead of waiters table.**

**Prompt Updation**

In [19]:
agent.run("In the WAITERS table only, group by waiter_name and count receipts. Don't use the receipts table.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ In the WAITERS table only, group by waiter_name and count receipts. Don't use the receipts table.               │
│                                                                                                                 │
╰─ InferenceClientModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = "SELECT waiter_name, COUNT(*) AS receipt_count FROM WAITERS GROUP BY waiter_name;"                       
  result = sql_engine(query)                                                                                       
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

('Canary Mill', 1)
('Corey Johnson', 1)
('Margaret James', 1)
('Michael Wantts', 1)
('Michael Watts', 1)
('Paul Walton', 1)

Out: None

[Step 1: Duration 2.42 seconds| Input tokens: 2,085 | Output tokens: 107]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_result = {                                                                                                 
      "Canary Mill": 1,                                                                                            
      "Corey Johnson": 1,                                                                                          
      "Margaret James": 1,                                                                                         
      "Michael Wantts": 1,                                                                                         
      "Michael Watts": 1,                                                                                          
      "Paul Walton": 1                                                                                             
  }                                                                                                                
  final_answer(final_result)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'Canary Mill': 1, 'Corey Johnson': 1, 'Margaret James': 1, 'Michael Wantts': 1, 'Michael Watts': 1, 
'Paul Walton': 1}

[Step 2: Duration 2.37 seconds| Input tokens: 4,410 | Output tokens: 230]

{'Canary Mill': 1,
 'Corey Johnson': 1,
 'Margaret James': 1,
 'Michael Wantts': 1,
 'Michael Watts': 1,
 'Paul Walton': 1}

In [20]:
agent.run("Show tip percentage per customer and rank them.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Show tip percentage per customer and rank them.                                                                 │
│                                                                                                                 │
╰─ InferenceClientModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = "SELECT customer_name, price, tip FROM receipts"                                                         
  data = sql_engine(query)                                                                                         
  print(data)                                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

('Alan Payne', 12.06, 1.2)
('Alex Mason', 23.86, 0.24)
('Woodrow Wilson', 53.43, 5.43)
('Margaret James', 21.11, 1.0)

Out: None

[Step 1: Duration 4.64 seconds| Input tokens: 2,073 | Output tokens: 150]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_ranked_tip_percentages = [                                                                                 
      ('Woodrow Wilson', 10.162829870859067),                                                                      
      ('Alan Payne', 9.950248756218905),                                                                           
      ('Margaret James', 4.737091425864519),                                                                       
      ('Alex Mason', 1.0058675607711651)                                                                           
  ]                                                                                                                
  final_answer(final_ranked_tip_percentages)                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: [('Woodrow Wilson', 10.162829870859067), ('Alan Payne', 9.950248756218905), ('Margaret James', 
4.737091425864519), ('Alex Mason', 1.0058675607711651)]

[Step 3: Duration 3.42 seconds| Input tokens: 7,343 | Output tokens: 555]

[('Woodrow Wilson', 10.162829870859067),
 ('Alan Payne', 9.950248756218905),
 ('Margaret James', 4.737091425864519),
 ('Alex Mason', 1.0058675607711651)]

Since this request is a bit harder than the previous one, we’ll switch the LLM engine to use the more powerful [Qwen/Qwen3-Next-80B-A3B-Thinking](https://huggingface.co/Qwen/Qwen3-Next-80B-A3B-Thinking)!

In [ ]:
sql_engine.description = updated_description

model = InferenceClientModel(
    provider="openai",
    api_key=os.environ["OPENAI_API_KEY"],
    model_id="gpt-4o-mini"
)
agent = CodeAgent(
    tools=[sql_engine],
    model=model,
)

agent.run("Which waiter got more total money from tips?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Which waiter got more total money from tips?                                                                    │
│                                                                                                                 │
╰─ InferenceClientModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = "SELECT customer_name, SUM(tip) as total_tips FROM receipts GROUP BY customer_name ORDER BY total_tips   
  DESC LIMIT 1;"                                                                                                   
  result = sql_engine(query)                                                                                       
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

('Woodrow Wilson', 5.43)

Out: None

[Step 1: Duration 2.86 seconds| Input tokens: 2,079 | Output tokens: 174]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Woodrow Wilson")                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Woodrow Wilson

[Step 2: Duration 1.51 seconds| Input tokens: 4,440 | Output tokens: 230]

'Woodrow Wilson'